In [1]:
from crypto_helpers import convert_numeric_to_ethereum_hash
from datetime import datetime
from synthetix_v3.base_perps import TestBasePerps
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import polars as pl

pl.Config.set_fmt_str_lengths(200)

polars.config.Config

In [2]:
base_perps = TestBasePerps()

In [3]:
# start and end blocks for the competition
start_block = 10536192
end_block = 11455992

start_timestamp = 1696260000
end_timestamp = 1698105600

In [4]:
settled_orders_df = base_perps.get_settled_orders(start_block=start_block, end_block=end_block) # https://goerli.basescan.org/block/10536192 # starting block time Monday, October 2, 2023 5:00:00 PM

In [5]:
polars_df = pl.from_pandas(settled_orders_df)

In [6]:
cleaned_settled_orders = (polars_df.select([
    pl.col('orderSettleds_timestamp'),
    pl.col('markets_marketSymbol'),
    pl.col('orderSettleds_accountId'),
    pl.col('orderSettleds_fillPrice'),
    pl.col('orderSettleds_accruedFunding'),
    pl.col('orderSettleds_sizeDelta'),
    pl.col('orderSettleds_newSize'),
    pl.col('orderSettleds_totalFees'),
])
.with_columns([
    pl.from_epoch("orderSettleds_timestamp").alias("datetime"),
    (pl.col('orderSettleds_fillPrice') / 10 ** 18),
    (pl.col('orderSettleds_sizeDelta') / 10 ** 18),
    (pl.col('orderSettleds_accruedFunding') / 10 ** 18),
    (pl.col('orderSettleds_newSize') / 10 ** 18),
    (pl.col('orderSettleds_totalFees') / 10 ** 18),
])
.with_columns([
    (pl.col('orderSettleds_sizeDelta').abs() * pl.col('orderSettleds_fillPrice')).alias('size_usd')
]))

In [7]:
cleaned_settled_orders.head(5)

orderSettleds_timestamp,markets_marketSymbol,orderSettleds_accountId,orderSettleds_fillPrice,orderSettleds_accruedFunding,orderSettleds_sizeDelta,orderSettleds_newSize,orderSettleds_totalFees,datetime,size_usd
i64,str,f64,f64,f64,f64,f64,f64,datetime[μs],f64
1697020928,"""ETH""",1.1224e9,1577.801896,-44.243833,86.997996,86.997996,68.632802,2023-10-11 10:42:08,137265.603517
1697023666,"""ETH""",1.1224e9,1572.851923,-0.229878,1.0,87.997996,0.31457,2023-10-11 11:27:46,1572.851923
1697023732,"""ETH""",1.1224e9,1572.968366,-0.005573,10.0,97.997996,3.145937,2023-10-11 11:28:52,15729.68366
1697023834,"""ETH""",1.1224e9,1573.181377,-0.008599,-97.997996,0.0,77.084311,2023-10-11 11:30:34,154168.622772
1697023922,"""ETH""",1.1224e9,1572.327512,-0.007403,3.283109,3.283109,1.032424,2023-10-11 11:32:02,5162.12198


### Liquidations

In [8]:
market_price_df = base_perps.get_market_updates()

In [9]:
pos_liqs_df = base_perps.get_position_liquidations()

In [10]:
position_liquidations_df = pl.from_pandas(pos_liqs_df)

In [11]:
# normalize numbers
market_price_polars_df = pl.from_pandas(market_price_df).with_columns([
    pl.from_epoch('marketUpdateds_timestamp').dt.round("1s").alias('datetime'),
    pl.col('marketUpdateds_price') / 10**18,
    pl.col('marketUpdateds_skew') / 10**18,
    pl.col('marketUpdateds_size') / 10**18,
    pl.col('marketUpdateds_sizeDelta') / 10**18,
    pl.col('marketUpdateds_currentFundingRate') / 10**18,
    pl.col('marketUpdateds_currentFundingVelocity') / 10**18
])

# normalize numbers
position_liquidations_df = position_liquidations_df.with_columns([
    pl.from_epoch('positionLiquidateds_timestamp').dt.round("1s").alias('datetime'),
    pl.col('positionLiquidateds_amountLiquidated') / 10**18,
    pl.col('positionLiquidateds_currentPositionSize') / 10**18,
    ]).filter(pl.col("datetime") > datetime(2023, 10, 2))

In [12]:
position_liquidations_usd_df = position_liquidations_df.join(market_price_polars_df, on='datetime', how='left')

In [13]:
position_liquidations_usd_df.head(5)

positionLiquidateds_id,positionLiquidateds_timestamp,positionLiquidateds_accountId,markets_id,positionLiquidateds_amountLiquidated,positionLiquidateds_currentPositionSize,markets_marketSymbol,datetime,marketUpdateds_timestamp,markets_id_right,marketUpdateds_price,marketUpdateds_skew,marketUpdateds_size,marketUpdateds_sizeDelta,marketUpdateds_currentFundingRate,marketUpdateds_currentFundingVelocity,markets_marketSymbol_right
str,i64,f64,i64,f64,f64,str,datetime[μs],i64,i64,f64,f64,f64,f64,f64,f64,str
"""200-1843551657-11725044""",1698643704,1.8436e9,200,0.116797,0.0,"""BTC""",2023-10-30 05:28:24,1698643704,200,34249.857322,190.210797,219.100197,-0.116797,0.02038,0.001712,"""BTC"""
"""100-170141183460469231731687303715884105881-11628833""",1698451282,1.7014e38,100,0.1927,0.0,"""ETH""",2023-10-28 00:01:22,1698451282,100,1779.583065,-743.462063,1260.523078,-0.1927,0.110394,-0.006691,"""ETH"""
"""100-1-11598645""",1698390906,1.0,100,0.11,0.0,"""ETH""",2023-10-27 07:15:06,1698390906,100,1792.209981,-743.461501,1260.522516,-0.11,0.115068,-0.006691,"""ETH"""
"""100-1570425441-11570247""",1698334110,1.5704e9,100,0.915089,0.0,"""ETH""",2023-10-26 15:28:30,1698334110,100,1769.11125,-743.351501,1260.632516,-0.915089,0.119466,-0.00669,"""ETH"""
"""200-170141183460469231731687303715884105780-11568784""",1698331184,1.7014e38,200,300.0,0.0,"""BTC""",2023-10-26 14:39:44,1698331184,200,33907.75,190.210737,219.100137,-300.0,0.014185,0.001712,"""BTC"""


In [14]:
position_liquidations_usd_df.columns

['positionLiquidateds_id',
 'positionLiquidateds_timestamp',
 'positionLiquidateds_accountId',
 'markets_id',
 'positionLiquidateds_amountLiquidated',
 'positionLiquidateds_currentPositionSize',
 'markets_marketSymbol',
 'datetime',
 'marketUpdateds_timestamp',
 'markets_id_right',
 'marketUpdateds_price',
 'marketUpdateds_skew',
 'marketUpdateds_size',
 'marketUpdateds_sizeDelta',
 'marketUpdateds_currentFundingRate',
 'marketUpdateds_currentFundingVelocity',
 'markets_marketSymbol_right']

In [15]:
cleaned_settled_orders.columns

['orderSettleds_timestamp',
 'markets_marketSymbol',
 'orderSettleds_accountId',
 'orderSettleds_fillPrice',
 'orderSettleds_accruedFunding',
 'orderSettleds_sizeDelta',
 'orderSettleds_newSize',
 'orderSettleds_totalFees',
 'datetime',
 'size_usd']

In [16]:
# rename columns for concatting
position_liq_final = position_liquidations_usd_df.rename({
    'positionLiquidateds_accountId': 'accountId', 
    'markets_marketSymbol_right': 'marketSymbol',
    'positionLiquidateds_amountLiquidated': 'size_delta',
    'positionLiquidateds_currentPositionSize': 'size',
    'marketUpdateds_price': 'price'
     }).with_columns([
    pl.lit(True).alias('liquidation')
]).select('datetime', 'accountId', 'marketSymbol', 'size_delta', 'size', 'price', 'liquidation')

settled_order_final = cleaned_settled_orders.rename({
    'orderSettleds_accountId': 'accountId', 
    'markets_marketSymbol': 'marketSymbol',
    'orderSettleds_sizeDelta': 'size_delta',
    'orderSettleds_newSize': 'size',
    'orderSettleds_fillPrice': 'price',
     }).with_columns([
    pl.lit(False).alias('liquidation')]).select('datetime', 'accountId', 'marketSymbol', 'size_delta', 'size', 'price', 'liquidation')

In [17]:
# get the overlap between unique accuonts
unique_liq_accts = position_liq_final['accountId'].unique().to_list()
unique_orders_accts = settled_order_final['accountId'].unique().to_list()

print(len(set(unique_liq_accts)))
print(len(set(unique_orders_accts)))

30
49


In [18]:
# are there liquidation accounts that are not in unique order accts?
print(len(set(unique_liq_accts) - set(unique_orders_accts)))


4


### Concat data and analyze results

In [19]:
concat_df = pl.concat([position_liq_final, settled_order_final])

In [21]:
concat_df = concat_df.with_columns(
        pl.col("accountId").map_elements(lambda val: convert_numeric_to_ethereum_hash(val), return_dtype=str),
    )

ColumnNotFoundError: orderSettleds_accountId

Error originated just after this operation:
DF ["datetime", "accountId", "marketSymbol", "size_delta"]; PROJECT */7 COLUMNS; SELECTION: "None"

In [ ]:
concat_df.filter(pl.col('liquidation') == True).shape

(385, 7)

In [ ]:
concat_df.filter(pl.col('liquidation') == False).shape

(2168, 7)

In [ ]:
filtered_acct = concat_df['accountId'][0]

In [ ]:
expanded_number = f'{filtered_acct:.0f}'
print(expanded_number)

170141183460469231731687303715884105728


In [ ]:
settled_order_final.filter(pl.col('accountId') == filtered_acct).filter(pl.col('marketSymbol') == 'ETH').sort(by='datetime', descending=False)

datetime,accountId,marketSymbol,size_delta,size,price,liquidation
datetime[μs],f64,str,f64,f64,f64,bool
2023-10-02 17:13:58,1.7014e38,"""ETH""",1.1885,1.1885,1682.513104,false
2023-10-02 17:22:50,1.7014e38,"""ETH""",95.773,95.773,1685.827036,false
2023-10-02 18:05:04,1.7014e38,"""ETH""",0.7604,1.6502,1689.579753,false
2023-10-02 18:08:06,1.7014e38,"""ETH""",97.1794,97.1794,1689.177899,false
2023-10-02 18:22:50,1.7014e38,"""ETH""",-1972.852,-1972.852,1688.204523,false
2023-10-02 18:33:26,1.7014e38,"""ETH""",-1977.15,-1977.15,1681.537133,false
2023-10-02 18:40:28,1.7014e38,"""ETH""",1977.15,0.0,1649.027979,false
2023-10-02 18:49:24,1.7014e38,"""ETH""",-1205.5381,-3178.3901,1656.602598,false
2023-10-02 19:05:58,1.7014e38,"""ETH""",604.4067,604.4067,1647.529609,false


In [ ]:
concat_df.filter(pl.col('accountId') == filtered_acct).filter(pl.col('marketSymbol') == 'ETH').sort(by='datetime', descending=False).tail(20)

datetime,accountId,marketSymbol,size_delta,size,price,liquidation
datetime[μs],f64,str,f64,f64,f64,bool
2023-10-19 20:59:28,1.7014e38,"""ETH""",319.217,638.7895,1561.342991,false
2023-10-20 01:12:50,1.7014e38,"""ETH""",639.8766,639.8766,1558.391228,false
2023-10-20 02:33:12,1.7014e38,"""ETH""",6499.9063,0.0,1579.435588,false
2023-10-20 05:34:12,1.7014e38,"""ETH""",-639.8766,0.0,1587.006112,false
2023-10-20 07:05:54,1.7014e38,"""ETH""",-946.7614,-946.7614,1589.34842,false
2023-10-20 08:13:54,1.7014e38,"""ETH""",-2209.2858,-2209.2858,1592.012076,false
2023-10-20 08:17:12,1.7014e38,"""ETH""",-638.7895,0.0,1591.136747,false
2023-10-20 08:55:38,1.7014e38,"""ETH""",-1552.9103,-2499.6717,1611.78725,false
2023-10-20 09:57:42,1.7014e38,"""ETH""",308.9051,308.9051,1615.990389,false
